In [37]:
"""
Monarch Mayfly Optimization (MMO) Algorithm for Feature Selection
=================================================================
This implementation of the Monarch Mayfly Optimization (MMO) algorithm
is a hybrid of the Monarch Butterfly Optimization (MBO) and the Mayfly Algorithm (MA).
Sources and Acknowledgements:
-----------------------------
1. Monarch Butterfly Optimization (MBO):
- Original implementation in Python by Justin van Zyl.
- Based on the study:
Wang G., Deb S., Cui Z., "Monarch Butterfly Optimization," Neural Comput & Applic 31:1995-2014.
doi: 10.1007/s00521-015-1923-y.
- Key operations utilized: Migration Operator, Adjusting Operator, and Elitism.
2. Mayfly Algorithm (MA):
- Extracted from the hybrid feature selection study:
Bhattacharyya, T., Chatterjee, B., Singh, P. K., Yoon, J. H., Geem, Z. W., & Sarkar, R. (2020).
"Mayfly in harmony: A new hybrid meta-heuristic feature selection algorithm," IEEE Access, 8, 195929-195945.
- The study hybridized MA with Harmony Search (HS). The MA part is used in this MMO hybrid.
Disclaimer:
-----------
This code is a hybrid implementation of the aforementioned algorithms and combines elements from both
to create the MMO algorithm for the purpose of feature selection. Full credit goes to the original authors
for their contributions.
"""

'\nMonarch Mayfly Optimization (MMO) Algorithm for Feature Selection\n=================================================================\nThis implementation of the Monarch Mayfly Optimization (MMO) algorithm\nis a hybrid of the Monarch Butterfly Optimization (MBO) and the Mayfly Algorithm (MA).\nSources and Acknowledgements:\n-----------------------------\n1. Monarch Butterfly Optimization (MBO):\n- Original implementation in Python by Justin van Zyl.\n- Based on the study:\nWang G., Deb S., Cui Z., "Monarch Butterfly Optimization," Neural Comput & Applic 31:1995-2014.\ndoi: 10.1007/s00521-015-1923-y.\n- Key operations utilized: Migration Operator, Adjusting Operator, and Elitism.\n2. Mayfly Algorithm (MA):\n- Extracted from the hybrid feature selection study:\nBhattacharyya, T., Chatterjee, B., Singh, P. K., Yoon, J. H., Geem, Z. W., & Sarkar, R. (2020).\n"Mayfly in harmony: A new hybrid meta-heuristic feature selection algorithm," IEEE Access, 8, 195929-195945.\n- The study hybridize

In [38]:
import numpy as np
import pandas as pd
import math
import random
from statistics import stdev
from time import process_time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [39]:
start_time = process_time()
np.random.seed(42)

In [40]:
# Load dataset
df = pd.read_csv('datasets\Breastcancer.csv')
tot_features = len(df.columns) - 1
x = df[df.columns[:tot_features]]
y = df[df.columns[-1]]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)

In [41]:
# Train classifier using original dataset
_classifier = SVC()
_classifier.fit(x_train, y_train)
predictions = _classifier.predict(x_test)
total_acc = accuracy_score(y_true=y_test, y_pred=predictions)
total_error = 1 - total_acc
total_features = tot_features
total_acc

0.5642857142857143

In [42]:
# Controlling parameters
swarm_size = 20
max_iterations = 20
alpha = 0.01
a1 = 3
a2 = 3.5
beta = 0.1
d = 3
fl = 3
l = 0.95
g = 1
delta = 0.9
lf_size = 1
adjusting_rate = 0.1
p = float(6/12)
s_max = 0.02
gmax=9.8
gmin=6
max_neighbors = 20

In [43]:
# Population structure and Initialization
subpop_size = swarm_size // 2
NP1_male_swarm_vel = np.zeros((subpop_size, tot_features))
NP1_female_swarm_vel = np.zeros((subpop_size, tot_features))
NP2_male_swarm_vel = np.zeros((subpop_size, tot_features))
NP2_female_swarm_vel = np.zeros((subpop_size, tot_features))

NP1_male_swarm_pos = np.random.uniform(low=-1, high=1, size=(subpop_size, tot_features))
NP1_female_swarm_pos = np.random.uniform(low=-1, high=1, size=(subpop_size, tot_features))
NP2_male_swarm_pos = np.random.uniform(low=-1, high=1, size=(subpop_size, tot_features))
NP2_female_swarm_pos = np.random.uniform(low=-1, high=1, size=(subpop_size, tot_features))

gbest_fitness = 1000000
pbest_fitness = np.empty(swarm_size)
pbest_fitness.fill(np.inf)
pbest = np.zeros((swarm_size, tot_features))
gbest = np.zeros(tot_features)
NP1_male_fitness = np.empty(subpop_size)
NP1_female_fitness = np.empty(subpop_size)
NP2_male_fitness = np.empty(subpop_size)
NP2_female_fitness = np.empty(subpop_size)
NP1_vmax_male = np.empty(tot_features)
NP1_vmax_female = np.empty(tot_features)
NP2_vmax_male = np.empty(tot_features)
NP2_vmax_female = np.empty(tot_features)

In [44]:
# S-shaped transfer function
def transfer_func(velocity):
    s1 = np.abs(velocity) * 0.5 + 1
    s1 = (-velocity) / s1 + 0.5
    return s1

In [45]:
# Fitness function
def find_fitness(particle):
    features = [df.columns[i] for i, v in enumerate(transfer_func(particle)) if v >= 0.25]
    if not features:
        return 10000
    new_x_train = x_train[features].copy()
    new_x_test = x_test[features].copy()
    _classifier = SVC()
    _classifier.fit(new_x_train, y_train)
    predictions = _classifier.predict(new_x_test)
    acc = accuracy_score(y_true=y_test, y_pred=predictions)
    err = 1 - acc
    num_features = len(features)
    fitness = alpha * (num_features / total_features) + (1 - alpha) * err
    return fitness

In [46]:
# Levy Flight function
def levy_flight(size):
    return np.sum(np.tan(math.pi * np.random.uniform(low=0, high=1, size=(1, size))))

In [47]:
def migration_operator(migrant_male_swarm_pos, female_swarm_pos, male_swarm_pos, peri, p):
    D = len(migrant_male_swarm_pos[0]) # Assuming all butterflies have the same dimensionality D
    
    for i in range(subpop_size):
        # Evaluate fitness for the current butterfly
        current_fitness = find_fitness(migrant_male_swarm_pos[i][1:])

        for k in range(1, D): # Starting from 1 as the 0th element is skipped (fitness)
            rand = np.random.uniform(low=0, high=1)
            r = rand * peri
            if r <= p:
                random_female_index = np.random.randint(0, subpop_size)
                selected_butterfly = female_swarm_pos[random_female_index]
            else:
                random_female_index = np.random.randint(0, subpop_size)
                selected_butterfly = male_swarm_pos[random_female_index]

            # Generate the kth element of the new butterfly
            migrant_male_swarm_pos[i][k] = selected_butterfly[k]

        # Evaluate fitness for the new butterfly
        new_fitness = find_fitness(migrant_male_swarm_pos[i][1:])

        # If the new fitness is better, update the 0th element of the butterfly
        if new_fitness < current_fitness:
            migrant_male_swarm_pos[i][0] = new_fitness

        # Print relevant information for debugging
        #print(f"For mayfly {i}: Current Fitness: {current_fitness}, New Fitness: {new_fitness}")

    return migrant_male_swarm_pos

In [48]:
def adjusting_operator(female_swarm_pos, male_swarm_pos, t):
    t += 1
    for i in range(subpop_size):
        if np.random.rand() < adjusting_rate:
            rand = np.random.uniform(low=0, high=1)
            if rand <= p:
                # Best butterfly will be the first, irrespective of NP designation
                if female_swarm_pos[0][0] <= male_swarm_pos[0][0]:
                    selected_butterfly = female_swarm_pos[0]
                else:
                    selected_butterfly = male_swarm_pos[0]

                levy_factor = s_max / (t**2)
                # Perform Levy flight perturbation on the selected butterfly
                for k in range(1, len(selected_butterfly)):
                    selected_butterfly[k] += levy_factor * (levy_flight(lf_size) - 0.5)
                    #selected_butterfly[k] += levy_factor * (levy_flight(selected_butterfly[k], lf_size) - 0.5)

                # Check if the selected butterfly has better fitness
                selected_fitness = find_fitness(selected_butterfly[1:])
                current_fitness = find_fitness(female_swarm_pos[i][1:])
                
                if selected_fitness < current_fitness:
                    # Replace a butterfly in the current population with the selected one
                    female_swarm_pos[i] = selected_butterfly.copy()
            else:
                # Randomly select a butterfly from the opposite gender
                random_male_or_female_index = np.random.randint(0, subpop_size)
                selected_butterfly = male_swarm_pos[random_male_or_female_index]

                # Check if the selected butterfly has better fitness
                selected_fitness = find_fitness(selected_butterfly[1:])
                current_fitness = find_fitness(female_swarm_pos[i][1:])

                if selected_fitness < current_fitness:
                    # Replace a butterfly in the current population with the selected one
                    female_swarm_pos[i] = selected_butterfly.copy()
    return female_swarm_pos

In [49]:
def update_vmax(swarm_pos_male, swarm_pos_female, vmax_male, vmax_female, subpop_size):
    for j in range(len(vmax_male)):
        r = np.random.normal(0, 1)
        index_male = min(subpop_size - 1, len(swarm_pos_male) - 1)
        index_female = min(subpop_size - 1, len(swarm_pos_female) - 1)

        vmax_male[j] = (swarm_pos_male[0][j] - swarm_pos_male[index_male][j]) * r
        vmax_female[j] = (swarm_pos_female[0][j] - swarm_pos_female[index_female][j]) * r

In [50]:
def sort_population(population_pos, population_vel):
    # Calculate fitness for each individual
    population_fitness = np.array([find_fitness(individual) for individual in population_pos])
    
    # Sort the population based on fitness
    sort_order = np.argsort(population_fitness)
    population_fitness = population_fitness[sort_order]
    population_pos = population_pos[sort_order]
    population_vel = population_vel[sort_order]

    return population_fitness, population_pos, population_vel

In [51]:
def update_gbest(gbest_fitness, gbest, male_fitness, male_swarm_pos, female_fitness, female_swarm_pos):
    if male_fitness[0] < gbest_fitness:
        gbest_fitness = male_fitness[0]
        gbest = male_swarm_pos[0].copy()

    if female_fitness[0] < gbest_fitness:
        gbest_fitness = female_fitness[0]
        gbest = female_swarm_pos[0].copy()

    return gbest_fitness, gbest

In [52]:
def crossover_and_mutation(NP_male_swarm_pos, NP_female_swarm_pos, NP_male_swarm_vel, NP_female_swarm_vel, tot_features,
subpop_size, l):
    NP_offspring1 = np.zeros((subpop_size, tot_features))
    NP_offspring2 = np.zeros((subpop_size, tot_features))

    for i in range(subpop_size):
        # Crossover
        partition = np.random.randint(tot_features // 4, math.floor((3 * tot_features // 4) + 1))
        for j in range(tot_features):
            NP_offspring1[i][j] = l * NP_male_swarm_pos[i][j] + (1 - l) * NP_female_swarm_pos[i][j]
            NP_offspring2[i][j] = l * NP_female_swarm_pos[i][j] + (1 - l) * NP_male_swarm_pos[i][j]
        
        if np.random.random() >= 0.5:
            NP_male_swarm_pos[i] = NP_offspring1[i].copy()
            NP_female_swarm_pos[i] = NP_offspring2[i].copy()
        else:
            NP_male_swarm_pos[i] = NP_offspring2[i].copy()
            NP_female_swarm_pos[i] = NP_offspring1[i].copy()

        # Mutation
        mutation_strength = 1.2
        r = np.random.exponential(scale=mutation_strength, size=tot_features)
        for j in range(tot_features):
            NP_male_swarm_pos[i][j] += r[j]
            NP_female_swarm_pos[i][j] += r[j]

        # Reset velocities
        NP_male_swarm_vel[i] = np.zeros(tot_features)
        NP_female_swarm_vel[i] = np.zeros(tot_features)

    return NP_male_swarm_pos, NP_female_swarm_pos, NP_male_swarm_vel, NP_female_swarm_vel

In [53]:
def update_swarm_position(swarm_pos, swarm_vel):
    for i in range(len(swarm_pos)):
        for j in range(len(swarm_pos[i])):
            swarm_pos[i][j] += swarm_vel[i][j]

In [54]:
def elitism(population_male_swarm_pos, population_male_swarm_vel, population_female_swarm_pos, 
            population_female_swarm_vel, elite_swarm_pos, elite_swarm_fitness, num_elite, tot_features):
    # Update Elite Individuals
    if find_fitness(population_male_swarm_pos[0]) < elite_swarm_fitness[0]:
        elite_swarm_fitness[0] = find_fitness(population_male_swarm_pos[0])
        elite_swarm_pos[0] = population_male_swarm_pos[0].copy()

    if find_fitness(population_female_swarm_pos[0]) < elite_swarm_fitness[0]:
        elite_swarm_fitness[0] = find_fitness(population_female_swarm_pos[0])
        elite_swarm_pos[0] = population_female_swarm_pos[0].copy()

    # Replace the worst individuals with the elite individuals
    for i in range(num_elite):
        population_male_swarm_pos[-1-i] = elite_swarm_pos[i].copy()
        population_male_swarm_vel[-1-i] = np.zeros(tot_features)
        population_female_swarm_pos[-1-i] = elite_swarm_pos[i].copy()
        population_female_swarm_vel[-1-i] = np.zeros(tot_features)

In [55]:
# Sorting initially for NP1 males
NP1_male_fitness, NP1_male_swarm_pos, NP1_male_swarm_vel = sort_population(NP1_male_swarm_pos, NP1_male_swarm_vel)
# Sorting initially for NP1 females
NP1_female_fitness, NP1_female_swarm_pos, NP1_female_swarm_vel = sort_population(NP1_female_swarm_pos, NP1_female_swarm_vel)
# Sorting initially for NP2 males
NP2_male_fitness, NP2_male_swarm_pos, NP2_male_swarm_vel = sort_population(NP2_male_swarm_pos, NP2_male_swarm_vel)
# Sorting initially for NP2 females
NP2_female_fitness, NP2_female_swarm_pos, NP2_female_swarm_vel = sort_population(NP2_female_swarm_pos, NP2_female_swarm_vel)

In [56]:
# Gbest updation for NP1 and NP2
gbest_fitness, gbest = update_gbest(gbest_fitness, gbest, NP1_male_fitness, NP1_male_swarm_pos, NP1_female_fitness, NP1_female_swarm_pos)
gbest_fitness, gbest = update_gbest(gbest_fitness, gbest, NP2_male_fitness, NP2_male_swarm_pos, NP2_female_fitness, NP2_female_swarm_pos)

In [57]:
# Add a variable to store the elite individuals
num_elite = 1
elite_swarm_pos = np.zeros((num_elite, tot_features))
elite_swarm_fitness = np.empty(num_elite)
elite_swarm_fitness.fill(np.inf)

# Initialize an empty list to store the convergence curve
# mmo_convergence_curve = []

In [ ]:
# Main Monarch Mayfly Optimization
for itr in range(max_iterations):
    # Updating vmax (velocity limit) for NP1
    update_vmax(NP1_male_swarm_pos, NP1_female_swarm_pos, NP1_vmax_male, NP1_vmax_female, subpop_size)

    # Updating vmax (velocity limit) for NP2
    update_vmax(NP2_male_swarm_pos, NP2_female_swarm_pos, NP2_vmax_male, NP2_vmax_female, subpop_size)
    
    ####################################################### NP1 ########################################################

    # NP1 Males
    for i in range(subpop_size):
        if NP1_male_fitness[i] < gbest_fitness:
            gbest_fitness = NP1_male_fitness[i]
            gbest = NP1_male_swarm_pos[i].copy()

        if NP1_male_fitness[i] < pbest_fitness[i]:
            pbest_fitness[i] = NP1_male_fitness[i]
            pbest[i] = NP1_male_swarm_pos[i].copy()

        # Velocity updation for NP1 males
        if i == 0:
            for j in range(tot_features):
                NP1_male_swarm_vel[0][j] = NP1_male_swarm_vel[0][j] + d * np.random.uniform(-1, 1)
        else:
            sum = 0
            for j in range(tot_features):
                sum = sum + (NP1_male_swarm_pos[i][j] - gbest[j]) * (NP1_male_swarm_pos[i][j] - gbest[j])
            rg = math.sqrt(sum)
            sum = 0
            for j in range(tot_features):
                sum = sum + (NP1_male_swarm_pos[i][j] - pbest[i][j]) * (NP1_male_swarm_pos[i][j] - pbest[i][j])
            rp = math.sqrt(sum)
            for j in range(tot_features):
                NP1_male_swarm_vel[i][j] = g * NP1_male_swarm_vel[i][j] + a1 * math.exp(-beta * rp * rp) * (pbest[i][j] - NP1_male_swarm_pos[i][j]) + a2 * math.exp(-beta * rg * rg) * (gbest[j] - NP1_male_swarm_pos[i][j])

                # Include attraction towards personal best (pbest)
                NP1_male_swarm_vel[i][j] += a1 * (pbest[i][j] - NP1_male_swarm_pos[i][j])

                # Include attraction towards global best (gbest)
                NP1_male_swarm_vel[i][j] += a2 * (gbest[j] - NP1_male_swarm_pos[i][j])

        # Velocity updation for NP1 females
        if NP1_female_fitness[i] <= NP1_male_fitness[i]:
            sum = 0
            for j in range(tot_features):
                sum = sum + (NP1_male_swarm_pos[i][j] - NP1_female_swarm_pos[i][j]) * (NP1_male_swarm_pos[i][j] - NP1_female_swarm_pos[i][j])
            rmf = math.sqrt(sum)
            NP1_female_swarm_vel[i][j] = g * NP1_female_swarm_vel[i][j] + a2 * math.exp(-beta * rmf * rmf) * (NP1_male_swarm_pos[i][j] - NP1_female_swarm_pos[i][j])
        else:
            for j in range(tot_features):
                NP1_female_swarm_vel[i][j] = g * NP1_female_swarm_vel[i][j] + fl * np.random.uniform(-1, 1)


    # Levy flight for NP1 females
    for i in range(subpop_size):
        if NP1_female_fitness[i] > NP1_male_fitness[i]:
        # Employ Levy flight for exploration
            for j in range(tot_features):
                NP1_female_swarm_pos[i][j] += levy_flight(lf_size)
        else:
            # Perform...
            for j in range(tot_features):
                NP1_female_swarm_pos[i][j] = np.random.uniform(-1, 1)
                

    # Call migration operator for NP1 males
    NP1_male_swarm_pos = migration_operator(NP1_male_swarm_pos, NP1_female_swarm_pos, NP2_male_swarm_pos, 1.2, float(5/12))
    # Call adjusting operator for NP2 females
    NP1_female_swarm_pos = adjusting_operator(NP1_female_swarm_pos, NP2_male_swarm_pos, itr)


    # Sorting for NP1 males
    NP1_male_fitness, NP1_male_swarm_pos, NP1_male_swarm_vel = sort_population(NP1_male_swarm_pos, NP1_male_swarm_vel)
    # Sorting for NP1 females
    NP1_female_fitness, NP1_female_swarm_pos, NP1_female_swarm_vel = sort_population(NP1_female_swarm_pos, NP1_female_swarm_vel)
    
    # Gbest updation
    gbest_fitness, gbest = update_gbest(gbest_fitness, gbest, NP1_male_fitness, NP1_male_swarm_pos, NP1_female_fitness, NP1_female_swarm_pos)

    # Crossover and mutation for NP1
    NP1_male_swarm_pos, NP1_female_swarm_pos, NP1_male_swarm_vel, NP1_female_swarm_vel = crossover_and_mutation(NP1_male_swarm_pos, NP1_female_swarm_pos, NP1_male_swarm_vel, NP1_female_swarm_vel, tot_features, subpop_size, l)

    # Updating swarm position for NP1
    update_swarm_position(NP1_male_swarm_pos, NP1_male_swarm_vel)
    update_swarm_position(NP1_female_swarm_pos, NP1_female_swarm_vel)

    # ELITISM: Save the elite individuals and Replace the worst individuals with the elite individuals for NP1
    elitism(NP1_male_swarm_pos, NP1_male_swarm_vel, NP1_female_swarm_pos, NP1_female_swarm_vel, elite_swarm_pos, elite_swarm_fitness, num_elite, tot_features)
    
    # Updating gravity and nuptial dance for NP1
    d = d * delta
    fl = fl * delta

    # Gbest updation
    gbest_fitness, gbest = update_gbest(gbest_fitness, gbest, NP1_male_fitness, NP1_male_swarm_pos, NP1_female_fitness, NP1_female_swarm_pos)

    ####################################################### NP2 ########################################################

    # NP2 Males
    iterations_without_improvement = 0
    while iterations_without_improvement < max_neighbors:
        for i in range(subpop_size):
            if NP2_male_fitness[i] < gbest_fitness:
                gbest_fitness = NP2_male_fitness[i]
                gbest = NP2_male_swarm_pos[i].copy()

            if NP2_male_fitness[i] < pbest_fitness[i]:
                pbest_fitness[i] = NP2_male_fitness[i]
                pbest[i] = NP2_male_swarm_pos[i].copy()

            # Velocity updation for NP2 males
            if i == 0:
                for j in range(tot_features):
                    NP2_male_swarm_vel[0][j] = NP2_male_swarm_vel[0][j] + d * np.random.uniform(-1, 1)
            else:
                sum = 0
                for j in range(tot_features):
                    sum = sum + (NP2_male_swarm_pos[i][j] - gbest[j]) * (NP2_male_swarm_pos[i][j] - gbest[j])
                rg = math.sqrt(sum)
                sum = 0
                for j in range(tot_features):
                    sum = sum + (NP2_male_swarm_pos[i][j] - pbest[i][j]) * (NP2_male_swarm_pos[i][j] - pbest[i][j])
                rp = math.sqrt(sum)
                for j in range(tot_features):
                    NP2_male_swarm_vel[i][j] = g * NP2_male_swarm_vel[i][j] + a1 * math.exp(-beta * rp * rp) * (pbest[i][j] - NP2_male_swarm_pos[i][j]) + a2 * math.exp(-beta * rg * rg) * (gbest[j] - NP2_male_swarm_pos[i][j])

                    # Include attraction towards personal best (pbest)
                    NP2_male_swarm_vel[i][j] += a1 * (pbest[i][j] - NP2_male_swarm_pos[i][j])
                    
                    # Include attraction towards global best (gbest)
                    NP2_male_swarm_vel[i][j] += a2 * (gbest[j] - NP2_male_swarm_pos[i][j])

            # Velocity updation for NP2 females
            if NP2_female_fitness[i] <= NP2_male_fitness[i]:
                sum = 0
                for j in range(tot_features):
                    sum = sum + (NP2_male_swarm_pos[i][j] - NP2_female_swarm_pos[i][j]) * (NP2_male_swarm_pos[i][j] - NP2_female_swarm_pos[i][j])
                rmf = math.sqrt(sum)
                NP2_female_swarm_vel[i][j] = g * NP2_female_swarm_vel[i][j] + a2 * math.exp(-beta * rmf * rmf) * (NP2_male_swarm_pos[i][j] - NP2_female_swarm_pos[i][j])
            else:
                for j in range(tot_features):
                    NP2_female_swarm_vel[i][j] = g * NP2_female_swarm_vel[i][j] + fl * np.random.uniform(-1, 1)


        # Levy flight for NP2 females
        for i in range(subpop_size):
            if NP2_female_fitness[i] > NP2_male_fitness[i]:
                # Employ Levy flight for exploration
                for j in range(tot_features):
                    NP2_female_swarm_pos[i][j] += levy_flight(lf_size)
            else:
                # Perform...
                for j in range(tot_features):
                    NP2_female_swarm_pos[i][j] = np.random.uniform(-1, 1)

        # Call migration operator for NP2 males
        NP2_male_swarm_pos = migration_operator(NP2_male_swarm_pos, NP2_female_swarm_pos, NP1_male_swarm_pos, 1.2, float(5/12))
        # Call adjusting operator for NP2 females
        NP2_female_swarm_pos = adjusting_operator(NP2_female_swarm_pos, NP1_male_swarm_pos, itr)

        # Sorting and updating for NP2 males
        NP2_male_fitness, NP2_male_swarm_pos, NP2_male_swarm_vel = sort_population(NP2_male_swarm_pos, NP2_male_swarm_vel)

        # Sorting and updating for NP2 females
        NP2_female_fitness, NP2_female_swarm_pos, NP2_female_swarm_vel = sort_population(NP2_female_swarm_pos, NP2_female_swarm_vel)

        #Gbest updation for NP2
        new_gbest_fitness, new_gbest = update_gbest(gbest_fitness, gbest, NP2_male_fitness, NP2_male_swarm_pos, NP2_female_fitness, NP2_female_swarm_pos)

        ############# v2 ################################
        if new_gbest_fitness < gbest_fitness:
            gbest_fitness = new_gbest_fitness
            gbest = new_gbest.copy()
            print("Iteration:", itr, "GBest Fitness:", gbest_fitness)
            iterations_without_improvement = 0 # Reset the counter when a better solution is found
        else:
            print("Iteration:", itr, "New GBest Fitness:", new_gbest_fitness)
            iterations_without_improvement += 1 # Increment the counter when no improvement is found

        # Crossover and mutation for NP2
        NP2_male_swarm_pos, NP2_female_swarm_pos, NP2_male_swarm_vel, NP2_female_swarm_vel = crossover_and_mutation(NP2_male_swarm_pos, NP2_female_swarm_pos, NP2_male_swarm_vel, NP2_female_swarm_vel, tot_features, subpop_size, l)
        
        # Updating swarm position for NP2
        update_swarm_position(NP2_male_swarm_pos, NP2_male_swarm_vel)
        update_swarm_position(NP2_female_swarm_pos, NP2_female_swarm_vel)

        # ELITISM: Save the elite individuals and Replace the worst individuals with the elite individuals for NP2
        elitism(NP2_male_swarm_pos, NP2_male_swarm_vel, NP2_female_swarm_pos, NP2_female_swarm_vel, elite_swarm_pos, elite_swarm_fitness, num_elite, tot_features)

        # Updating gravity and nuptial dance for NP2
        g = gmax-((gmax-gmin)*itr/max_iterations)
        d = d * delta
        fl = fl * delta
        
        #Gbest updation for NP2
        gbest_fitness, gbest = update_gbest(gbest_fitness, gbest, NP2_male_fitness, NP2_male_swarm_pos, NP2_female_fitness, NP2_female_swarm_pos)
    
    # Append the current gbest_fitness to the convergence_curve list
    # mmo_convergence_curve.append(gbest_fitness)
    print("Iteration:", itr, "GBest Fitness:", gbest_fitness)

Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 New GBest Fitness: 0.013071428571428546
Iteration: 0 GBest Fitness: 0.012071428571428545
Iteration: 0 New GBest Fitness: 0.012071428571428545
Iteration: 0 New GBest Fitness: 0.012071428571428545
Iteration: 0 New GBest Fitness: 0.012071428571428545
Iteration: 0 New GBest Fitness: 0.012071428571428545
Iteration: 0 New GBest Fitness: 0.012071428571428545
Iteration: 0 New GBest Fitness: 0.0120714285714285

In [ ]:
# Gbest = transfer_func(gbest)
selected_features = transfer_func(gbest)
for j in range(tot_features):
    if selected_features[j] >= 0.25:
        selected_features[j] = 1
    else:
        selected_features[j] = 0
number_of_selected_features = np.sum(selected_features)
print("NUM:", number_of_selected_features)
print("SELECTED FEATURES:", selected_features)

In [ ]:
features = [df.columns[j] for j in range(len(selected_features)) if selected_features[j] == 1]
if not features:
    acc = 0
else:
   new_x_train = x_train[features]
    new_x_test = x_test[features]
    _classifier = SVC(random_state=seed)
    _classifier.fit(new_x_train, y_train)
    predictions = _classifier.predict(new_x_test)
    acc = accuracy_score(y_true=y_test, y_pred=predictions)
    fitness = acc
print("ACC:", acc)

In [ ]:
end_time=process_time()
print("time in seconds =",(end_time-start_time))
print("time in microseconds =",(end_time-start_time) * 1e6)